In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Abstract
This is a convolutional neural network that measures population density from birds-eye view images. It takes-in an input image and then detects the population density into 4 outputs:
- 0 people = Zero Density
- 1 to 5 people = Low Density
- 6 to 30 = Medium Density
- 31 to infinity = High Density

# Defining Constants
Let's define some constants to use in our neural network for later.

In [0]:
# import numpy
import numpy as np

# set the random number generator of numpy
from numpy.random import seed
seed(1)

In [0]:
# how aggressive will be the data augmentation / transformation
transformation_ratio = .05

In [0]:
# number of training and testing images
num_training_samples = 100
num_testing_samples = 60

# number of images to process before the weights are updated: try 4, 8, 16, 32, etc. depending on the CPU/GPU memory capacity
training_batch_size = 16
testing_batch_size = 8

# image's square dimension for the neural network (width x height)
image_size = 224

# steps is the number of images per epoch
training_steps = np.ceil(num_training_samples / training_batch_size)
testing_steps = np.ceil(num_testing_samples / testing_batch_size)

# Pre-Processing Image Data
Let's pre-process image data using Image Augmentation from Keras's ImageDataGenerator class.

Image augmentation allows us to create many batches of the images, which create many more diverse set of the images. Some augmentations could be rotating, stretching, zooming, etc.

This helps prevent overfitting because augmentation better diversifies the data set.

In [5]:
# import keras
import tensorflow.keras as keras

# seed the random number generator of tensorflow
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
# import the keras image data augmentor
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
# create a data generator for the training set
training_batches = ImageDataGenerator(
    rescale= 1.0 / 255,
    rotation_range = transformation_ratio,
    shear_range = transformation_ratio,
    zoom_range = transformation_ratio,
    cval = transformation_ratio,
    horizontal_flip=True,
    vertical_flip=True,
).flow_from_directory(
    "drive/My Drive/Flare/images/training",
    target_size = (image_size, image_size),
    batch_size = training_batch_size
)

Found 100 images belonging to 4 classes.


In [8]:
# create a data generator for the testing set using the built-in mobilenet functions
testing_batches = ImageDataGenerator(
    rescale = 1.0 / 255
).flow_from_directory(
    "drive/My Drive/Flare/images/testing",
    target_size = (image_size, image_size),
    batch_size = testing_batch_size
)

Found 60 images belonging to 4 classes.


# Convolutional Neural Network
Let's create a convolutional neural network to classify the image based on the categories.

In [0]:
# import the basic model class
from tensorflow.keras.models import Model

# import the xception model
from tensorflow.keras.applications.vgg16 import VGG16

# import layers
from tensorflow.keras.layers import GlobalMaxPooling2D, Dropout, Dense

In [10]:
# use the pre-tained VGG16 model
base_model = VGG16(input_shape = (image_size, image_size, 3), weights = "imagenet", include_top = False)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
# add a MaxPooling layer
pool_layer = GlobalMaxPooling2D()(base_model.output)

# add a Dropout layer to prevent overfitting
dropout_layer = Dropout(rate=0.20)(pool_layer)

# add the output layer to the neural network
output_layer = Dense(units = 4, activation = "softmax")(dropout_layer)
model = Model(base_model.input, output_layer)

In [0]:
# do not train the layers from the base (original) Xception model
for layer in base_model.layers:
    layer.trainable = False

In [0]:
# compile the model using the nadam (stochastic Gradient Descent) optimizer
model.compile(
    optimizer="nadam",
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

# Callbacks
Let's define Callbacks to save the neural network whenever we reached the most accurate version of the model.

In [0]:
# import the callbacks for the models to use to fit to the training set
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [0]:
# declare a checkpoint to save the best version of the model
model_file = "model.h5"
checkpoint = ModelCheckpoint(model_file, monitor = "val_acc", save_best_only = True, mode = "max")

# early stop the model as the validation accuracy stagnates
# early_stop = EarlyStopping(monitor = "val_acc", patience = 5, verbose = 0)

# reduce the learning rate as the learning stagnates
# reduce_lr = ReduceLROnPlateau(monitor="val_acc", factor=0.5, patience=2, mode="max", min_lr=0.00001)

callbacks_list = [checkpoint]

# Fitting The Model
Let's fit the model to the training set.

In [16]:
classifier = model.fit_generator(
    training_batches,
    steps_per_epoch = training_steps,
    validation_data = testing_batches,
    validation_steps = testing_steps,
    epochs = 100,
    verbose = 1,
    callbacks = callbacks_list
)

Epoch 1/100
7/7 [==============================] - 9s 1s/step - loss: 1.5527 - acc: 0.2900 - val_loss: 1.3442 - val_acc: 0.3500
Epoch 2/100
7/7 [==============================] - 3s 391ms/step - loss: 1.4917 - acc: 0.3700 - val_loss: 1.2740 - val_acc: 0.4000
Epoch 3/100
7/7 [==============================] - 3s 414ms/step - loss: 1.2737 - acc: 0.3700 - val_loss: 1.2316 - val_acc: 0.4667
Epoch 4/100
7/7 [==============================] - 3s 399ms/step - loss: 1.2986 - acc: 0.4400 - val_loss: 1.1979 - val_acc: 0.4667
Epoch 5/100
7/7 [==============================] - 3s 428ms/step - loss: 1.2339 - acc: 0.4800 - val_loss: 1.1888 - val_acc: 0.5000
Epoch 6/100
7/7 [==============================] - 3s 421ms/step - loss: 1.0566 - acc: 0.5400 - val_loss: 1.1298 - val_acc: 0.5500
Epoch 7/100
7/7 [==============================] - 3s 455ms/step - loss: 1.0767 - acc: 0.5300 - val_loss: 1.1596 - val_acc: 0.5833
Epoch 8/100
7/7 [==============================] - 3s 412ms/step - loss: 0.9934 - acc:

KeyboardInterrupt: ignored